# Intro to REST APIs

This notebook demonstrates how to create requests and parse responses for the Data and Orders API with a quick code snippet at the end dedicated to the subscritption API. We will create a search request for the Data API using  a `Geojson`. We will parse the response for image IDs. We will use those image IDs to place a order request.


More reference information can be found at [Ordering & Delivery](https://developers.planet.com/apis/orders/).

The following Python code is a series of import statements. These statements are used to include various Python modules and specific functions or classes from these modules into the current script. Here's a brief explanation of each import:

- `import json`: This imports the `json` module, which provides methods to manipulate JSON data, including methods for parsing JSON from strings and files, and for converting Python objects into JSON.

- `import os`: This imports the `os` module, which provides a way of using operating system dependent functionality, such as reading or writing to the environment, manipulating paths, and more.

- `import pathlib`: This imports the `pathlib` module, which offers classes representing filesystem paths with semantics appropriate for different operating systems.

- `import time`: This imports the `time` module, which provides various time-related functions.

- `from ipyleaflet import Map, Polygon`: This imports the `Map` and `Polygon` classes from the `ipyleaflet` module. `ipyleaflet` is a library for interactive maps, and `Map` and `Polygon` are classes used to create and manipulate map objects.

- `import requests`: This imports the `requests` module, which is used for making HTTP requests.

- `from requests.auth import HTTPBasicAuth`: This imports the `HTTPBasicAuth` class from the `requests.auth` module. This class is used to handle HTTP Basic Authentication.

In [ ]:
import json
import os
import pathlib
import time
from ipyleaflet import Map, Polygon

import requests
from requests.auth import HTTPBasicAuth

The following Python code is a command that installs two Python packages: `requests` and `ipyleaflet`.

This line of code is often used in Jupyter notebooks to ensure that the necessary Python packages are installed before the rest of the code in the notebook is run.

The `#!pip install` command is a way to run the pip package manager, which is used to install Python packages. The packages to be installed are listed after the `install` keyword.

In this case, the `requests` package is a popular Python library used for making HTTP requests. It abstracts the complexities of making requests behind a beautiful, simple API, allowing you to send HTTP/1.1 requests.

The `ipyleaflet` package is a library that provides interactive maps in Jupyter notebooks. It's a bridge between Jupyter and the Leaflet JavaScript library for creating interactive maps.

By running this command, both the `requests` and `ipyleaflet` packages will be installed in the current Python environment, making them available for import in this and other Python scripts.

In [ ]:
#!pip install requests ipyleaflet

The following Python code is a single line that prompts the user to input their Planet API key and assigns it to the variable `PLANET_API_KEY`.

The `input()` function in Python is used to collect user input from the console. The string argument provided to the `input()` function, "Paste your API key here: ", is displayed as a prompt to the user.

An API key is a token that a program sends to identify itself to an API (Application Programming Interface). The API key is used to track and control how the API is being used, for example to prevent malicious use or abuse of the API. In this case, the API key is used to authenticate requests to the Planet API.

After the user pastes their API key and presses Enter, the API key is stored as a string in the `PLANET_API_KEY` variable. This variable can then be used in subsequent API requests for authentication.

In [ ]:
# Get the API key from the user
PLANET_API_KEY = input("Paste your API key here: ")

PLANET_API_KEY # uncomment this line to print the API key, 
# uncomment # the above line to print the API key,for troubleshooting, but be sure to clear the output before sharing the notebook. 

The following Python code is defining two string variables: `orders_url` and `data_url`. These variables are assigned the URLs of two different APIs provided by Planet.

The `orders_url` variable is assigned the URL of Planet's Orders API v2. This API is used to place, update, and manage orders for Planet's data products. 

The `data_url` variable is assigned the URL of Planet's Data API v1. This API is used to search for and download Planet's satellite imagery and other geospatial data.

These variables can be used later in the code to make HTTP requests to these APIs. By storing the URLs in variables, it's easier to manage and update the URLs throughout the code. For example, if the version of the API changes, you only need to update the URL in one place.

In [ ]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2/'
data_url = "https://api.planet.com/data/v1"

The following Python code is making an authenticated HTTP GET request to a URL and storing the response.

The first line creates an instance of the `HTTPBasicAuth` class from the `requests` library, passing in the `PLANET_API_KEY` as the username and an empty string as the password. This `HTTPBasicAuth` object will be used to authenticate the HTTP request. HTTP Basic Authentication is a simple authentication scheme built into the HTTP protocol, where the client sends the username and password as Base64-encoded text.

The second line uses the `requests.get()` function to send an HTTP GET request to the URL stored in the `data_url` variable. The `auth` parameter is set to the `HTTPBasicAuth` object created in the previous line. This means the request will include the necessary headers for HTTP Basic Authentication.

The response from the server is stored in the `response` variable. This `Response` object contains the server's response to the request. You can access the response's content, status code, headers, and other information using the properties and methods of the `Response` object.

The third line simply evaluates the `response` variable. In a Jupyter notebook, this would display a string representation of the `Response` object, which includes the HTTP status code and the response's URL. To view the content of the response, you would use `response.content` or `response.text`. To parse the content as JSON, you would use `response.json()`.


In the following code, if the `PLANET_API_KEY` is valid and the server is functioning correctly, you should receive a `200 OK` response. If the API key is invalid, you might receive a `401 Unauthorized` response. If the `data_url` is incorrect, you might receive a `404 Not Found` response.

You can check the status code of the response in your Python code using `response.status_code`. If you want to raise an exception for unsuccessful status codes, you can use `response.raise_for_status()`.

In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(data_url, auth=auth)
response

## Searching with the Data API
We can use the [data API](https://developers.planet.com/docs/apis/data/) in order to automate searching based on the search criterias like: date range, cloud cover, area cover, aoi. We can create and AOI using https://geojson.io/

The following Python code defines two functions: `parse_geojson` and `map_polygon`.

The `parse_geojson` function takes a filename as an argument, which is expected to be a GeoJSON file. GeoJSON is a format for encoding geographic data structures. The function opens the file, loads the JSON data, and checks the "type" of the GeoJSON object. If the type is "FeatureCollection", it iterates over the features and extracts the geometry of each feature. If the type is "Feature", it directly extracts the geometry. If the type is neither "FeatureCollection" nor "Feature", it assumes the entire object is the geometry. The function then returns the geometry.

The `map_polygon` function takes a geometry object as an argument, which is expected to be a polygon. It first switches the coordinates from (latitude, longitude) to (longitude, latitude). Then it calculates the center of the polygon by averaging the latitudes and longitudes. It creates a `Map` object centered at the calculated center and with a zoom level of 14. It then creates a `Polygon` object with the switched coordinates, and with both the outline and fill color set to green. This polygon is added as a layer to the map. Finally, the function returns the map with the polygon layer.





In [ ]:
def parse_geojson(filename):
    features = json.load(open(filename))
    if "type" in features and features["type"] == "FeatureCollection":
        for f in features["features"]:
            geoms =  f["geometry"]
    elif "type" in features and features["type"] == "Feature":
        geoms= features["geometry"]
    else:
        geoms = features

    return geoms

def map_polygon(geom):
    # To switch to (longitude, latitude)
    polygon_coords = [(lon, lat) for lat, lon in geom['coordinates'][0]]
    
    # Calculate the center of your polygon
    latitudes = [p[0] for p in polygon_coords]
    longitudes = [p[1] for p in polygon_coords]
    
    # Map's center is the average of the polygon's coordinates
    center = (sum(latitudes) / len(polygon_coords), sum(longitudes) / len(polygon_coords))
    m = Map(center=center, zoom=14)

    # Create a polygon and add it to the map
    polygon = Polygon(
        locations=polygon_coords,
        color="green",
        fill_color="green"
    )
    m.add_layer(polygon)
    return m

The following Python code is using the `parse_geojson` and `map_polygon` functions to load a polygon from a GeoJSON file and display it on a map.

The first line calls the `parse_geojson` function with the filename of the GeoJSON file as an argument. This function opens the file, loads the JSON data, and extracts the geometry of the geographic feature(s) in the file. The geometry data is then stored in the `geometry` variable.

The GeoJSON file in this case is named "lake-lagunita-large.geojson". GeoJSON is a format for encoding geographic data structures, and this file presumably contains data for a large area of Lake Lagunita.

The second line calls the `map_polygon` function with the `geometry` variable as an argument. This function creates a map, creates a polygon from the geometry data, adds the polygon as a layer to the map, and then returns the map. The map is then displayed in the Jupyter notebook.

In summary, this code is loading geographic data from a GeoJSON file and displaying it on a map. The geographic feature being displayed is presumably a large area of Lake Lagunita.

In [ ]:
geometry = parse_geojson("lake-lagunita-large.geojson")
map_polygon(geometry)

The following Python code is creating a series of filters to be used when querying a geospatial database for images. These filters are defined as dictionaries, which is a common way to structure data in Python.

The `geometry_filter` is set up to filter images based on their geographic location. The `config` field is set to the `geometry` variable, which is expected to contain the geographic coordinates of the area of interest (AOI).

The `date_range_filter` is set up to filter images based on the date they were acquired. The `config` field specifies a date range using the "greater than or equal to" (`gte`) and "less than or equal to" (`lte`) operators. This filter will only return images that were acquired between October 7, 2023, and October 11, 2023.

The `cloud_cover_filter` is set up to filter images based on their cloud coverage. The `config` field specifies a maximum cloud coverage using the "less than or equal to" (`lte`) operator. This filter will only return images with a cloud coverage of 1 or less, which can be interpreted as 100% or less depending on the database's cloud coverage scale.

Finally, the `combined_filter` is set up to combine the previous three filters using an "And" operator. This means that only images that satisfy all three filters (overlap with the AOI, were acquired within the specified date range, and have a cloud coverage of 1 or less) will be returned.

These filters can be used in a query to a geospatial database to retrieve specific images. The exact way to use these filters will depend on the database's API.

In [ ]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte":"2023-10-07T00:00:00Z",
    "lte":"2023-10-11T00:00:00Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 1
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

The following Python code is creating a dictionary called `search_request` that is structured to be used as a request object for an API. This request object is designed to search for specific items in a geospatial database.

The `item_type` variable is set to "PSScene". This is likely a specific type of item or data product in the database. For example, in the Planet API, "PSScene" refers to a type of satellite imagery product.

The `search_request` dictionary is then created with two keys: `item_types` and `filter`. 

The `item_types` key is assigned a list containing the `item_type` variable. This means the request will search for items of type "PSScene".

The `filter` key is assigned the `combined_filter` variable, which is expected to be a dictionary containing filter criteria. This means the request will only return items that meet the criteria specified in `combined_filter`.

The last line simply evaluates the `search_request` variable. In a Jupyter notebook, this would display the dictionary.

In summary, this code is setting up a request to search a geospatial database for "PSScene" items that meet certain criteria. The exact way to send this request will depend on the database's API.

In [ ]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}


search_request

The following Python code is making a POST request to the Planet API to search for specific geospatial data, and then extracting and printing the IDs of the returned images.

The first part of the code sends a POST request to the 'https://api.planet.com/data/v1/quick-search' endpoint of the Planet API. The `requests.post()` function is used to send the request. The `auth` parameter is set to an instance of `HTTPBasicAuth` with the `PLANET_API_KEY` as the username and an empty string as the password, which is used for authentication. The `json` parameter is set to `search_request`, which is expected to be a dictionary containing the search parameters.

The response from the server is stored in the `search_result` variable. This `Response` object contains the server's response to the request.

The second part of the code extracts the IDs of the returned images. It first converts the response content to JSON using `search_result.json()`, then accesses the 'features' key of the JSON object, which is expected to be a list of features. For each feature in this list, it extracts the 'id' key, which is expected to be the ID of an image. These IDs are stored in the `image_ids` list.

Finally, the `print()` function is used to print the `image_ids` list. This will display the IDs of the images that were returned by the search request.

In summary, this code is using the Planet API to search for geospatial data that meet certain criteria, and then printing the IDs of the returned images.

In [ ]:
# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)

The followong Python code is calling the `map_polygon` function with a specific geometry from the results of a search request to an API.

The `search_result.json()` part of the code is converting the response from the API request, which is stored in `search_result`, into a Python dictionary.

The `['features'][1]['geometry']` part of the code is accessing the 'geometry' key of the second feature in the 'features' list of the dictionary. In the context of a geospatial API, 'features' typically refers to a list of geographic features, and 'geometry' typically refers to the geographic coordinates of a feature.

The `map_polygon` function is then called with this geometry as an argument. This function is expected to create a map and display the specified geometry as a polygon on the map.

In summary, this code is displaying the second geographic feature from the search results on a map. The exact appearance of the map will depend on the implementation of the `map_polygon` function.

In [ ]:
map_polygon(search_result.json()['features'][1]['geometry'])

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/apis/orders/reference/) AOI reference.

In [ ]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

## Ordering

In this example, we will order two `PSScene` analytic images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/apis/orders/scenes/).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic` bundle for the  `PSScene` item contains 3 assets: the analytic image, the analytic xml file, and the udm. See the [Product bundles reference](https://developers.planet.com/docs/orders/product-bundles-reference/) to learn about other bundles and other items.

Now we will list the names of orders we have created thus far. Your list may be empty if you have not created an order yet.

The following Python code is working with the response from a REST API request, specifically extracting data from the response and then processing it.

The first line of the code is converting the response from the API request, which is stored in the `response` variable, into a Python dictionary using the `json()` method. It then accesses the 'orders' key of the dictionary and assigns the corresponding value to the `orders` variable. The 'orders' key is expected to contain a list of orders.

The second line of the code is a list comprehension that iterates over the first five elements of the `orders` list (as specified by `orders[:5]`) and for each order (referred to as `r`), it extracts the 'name' key. The result is a list of the 'name' values of the first five orders.

In summary, this code is extracting the first five orders from the response of an API request and creating a list of their names. The exact meaning of 'orders' and 'name' will depend on the API and the data it provides.

In [ ]:
orders = response.json()['orders']
[r['name'] for r in orders[:5]]
orders

### Place Order

The following Python code is creating a dictionary called `headers` that sets the 'Content-Type' header to 'application/json'. This dictionary can be used in HTTP requests to specify the media type of the body of the request (or response).

The 'Content-Type' header is a standard HTTP header that indicates the media type of the resource in the body of the message. 'application/json' is a media type that indicates that the resource is a JSON (JavaScript Object Notation) document.

In the context of a REST API, when you send a request to the server, you often need to specify the format of the data you're sending. By setting the 'Content-Type' header to 'application/json', you're telling the server that you're sending JSON-formatted data.

In summary, this code is preparing a header that can be used in HTTP requests to indicate that the body of the request will be in JSON format.

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

The following Python code is defining three lists of dictionaries: `single_product`, `same_src_products`, and `multi_src_products`. Each dictionary represents a product that is part of an order.

Each dictionary in the lists has three keys: `item_ids`, `item_type`, and `product_bundle`.

- `item_ids` is a list of unique identifiers for the items in the product. These are likely specific data sets or images in the geospatial database.

- `item_type` is a string that specifies the type of the items. In this case, all items are of type "PSScene", which likely refers to a specific type of geospatial data.

- `product_bundle` is a string that specifies the type of product bundle. The product bundle determines what kind of processing has been done on the data. For example, "analytic_udm2" might refer to a bundle that includes analytic data and an Unusable Data Mask (UDM), while "visual" might refer to a bundle that includes data processed for visual use.

The `single_product` list contains one product with one item.

The `same_src_products` list contains one product with two items. These items likely come from the same source or represent the same area, given the name of the list.

The `multi_src_products` list contains two products, each with one item. These items likely come from different sources or represent different areas, given the name of the list.

In summary, this code is defining orders for the Planet data API. Each order consists of one or more products, and each product consists of one or more items of a specific type and product bundle.

In [ ]:
# define products part of order
single_product = [
    {
      "item_ids": ["20231008_180635_41_24bb"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

same_src_products = [
    {
      "item_ids": ["20151119_025740_0c74",
                   "20151119_025739_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

multi_src_products = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    },
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "visual"
    },
    
]

The selected Python code is creating two Python dictionaries: `product` and `request`. These dictionaries are structured to match the expected input for a geospatial data API.

The `product` dictionary represents a product that is part of an order. It contains three keys:

- `item_ids` is a list containing the first element of the `image_ids` list. This is likely a unique identifier for a specific geospatial dataset or image.
- `item_type` is a string that specifies the type of the item. In this case, the item is of type "PSScene", which likely refers to a specific type of geospatial data.
- `product_bundle` is a string that specifies the type of product bundle. In this case, the product bundle is "visual", which likely refers to data processed for visual use.

The `request` dictionary represents an order request. It contains three keys:

- `name` is a string that specifies the name of the order. In this case, the name is "Stanford demo".
- `products` is a list of products that are part of the order. In this case, the list contains the `product` dictionary defined earlier.
- `delivery` is a dictionary that specifies the delivery options for the order. The `single_archive` key is set to `True`, indicating that all products should be delivered in a single archive. The `archive_type` key is set to "zip", indicating that the archive should be a ZIP file.

In summary, this code is creating an order request for a geospatial data API. The order consists of a single product, which is a visual representation of a specific geospatial dataset. The order should be delivered as a single ZIP file.

In [ ]:
product = [
    {
      "item_ids": [image_ids[0]],
      "item_type": "PSScene",
      "product_bundle": "visual"
    }]




request = {  
   "name":"Stanford demo",
   "products": product,
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

The above Python code defines a function named `place_order` that takes two parameters: `request` and `auth`. This function is designed to place an order by making a POST request to a REST API.

The function begins by making a POST request to the URL stored in the `orders_url` variable. The `requests.post` method is used to send the request. The `data` parameter of this method is set to `json.dumps(request)`, which converts the `request` dictionary into a JSON-formatted string. The `auth` parameter is used for authentication, and the `headers` variable is used to set the HTTP headers for the request.

The response from the API request is stored in the `response` variable. The function then prints the JSON representation of the response using `response.json()`.

Next, the function extracts the 'id' from the JSON response and assigns it to the `order_id` variable. This 'id' is likely a unique identifier for the order that was just placed. The function then prints this order ID.

The function then constructs the URL for the order by appending the order ID to the `orders_url` string, with a '/' in between. This URL is likely the endpoint that can be used to access the details of the order.

Finally, the function returns the order URL.

In summary, this function places an order by sending a POST request to a REST API, prints the response and the order ID, constructs the order URL, and then returns this URL.

In [ ]:
order_url = place_order(request, auth)

The above Python code is calling the `place_order` function and assigning the result to the variable `order_url`.

The `place_order` function is expected to take two arguments: `request` and `auth`. 

- `request` is likely a dictionary that contains the details of the order to be placed. This could include information such as the products to be ordered, the delivery method, and the name of the order.

- `auth` is likely an object or a tuple that contains the authentication information required to place the order. This could be an API key, a username/password pair, or some other form of credentials.

The `place_order` function is expected to place the order and return the URL for the order. This URL is then stored in the `order_url` variable.

In summary, this line of code is placing an order by calling the `place_order` function with the specified request and authentication information, and storing the URL of the placed order.

### Poll for Order Success

The following Python code defines a function named `poll_for_success` and then calls this function. This function is designed to repeatedly check the status of an order by making GET requests to a REST API until the order reaches a certain state or a specified number of checks have been made.

The `poll_for_success` function takes three parameters: `order_url`, `auth`, and `num_loops`. 

- `order_url` is the URL for the order that should be checked.
- `auth` is the authentication information required to make the request.
- `num_loops` is the maximum number of times the order status should be checked. It defaults to 10 if no value is provided.

The function starts by initializing a counter variable `count` to 0. It then enters a while loop that continues until `count` is less than `num_loops`.

Inside the loop, the function increments `count` by 1 and then makes a GET request to `order_url` using the `requests.get` method. The `auth` parameter is used for authentication.

The response from the API request is converted into a Python dictionary using the `json()` method, and the 'state' of the order is extracted from this dictionary and printed.

The function then checks if the state of the order is in the list `end_states`, which contains the strings 'success', 'failed', and 'partial'. If the state is in this list, the function breaks out of the loop.

If the state is not in `end_states`, the function waits for 10 seconds before the next iteration of the loop. This is done using the `time.sleep(10)` statement.

After defining the function, the code calls `poll_for_success` with `order_url` and `auth` as arguments.

In summary, this function checks the state of an order a specified number of times, waiting 10 seconds between each check, and stops checking once the order reaches a certain state. The function is then called with a specific order URL and authentication information.

In [ ]:
def poll_for_success(order_url, auth, num_loops=10):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response[  'state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, auth)

## Bandmath and clip
Now lets use the toolchain in order to combine a few opperations at once.

To create a polygon to clip by lets go to [geojson.io](https://geojson.io)


In [ ]:
# define the clip tool
clip = {
    "clip": {
        "aoi": geometry
    }
}

The above Python code is defining a dictionary named `clip`. This dictionary is likely used to specify parameters for a geospatial data processing operation, specifically a clipping operation.

In the context of geospatial data, "clipping" refers to the process of cutting out a piece of a larger dataset that falls within a specified area of interest (AOI). The resulting "clip" includes only the data within the AOI.

The `clip` dictionary has a single key-value pair. The key is the string "clip", and the value is another dictionary.

This inner dictionary also has a single key-value pair. The key is the string "aoi", and the value is the variable `geometry`. The `geometry` variable is not defined in the selected code, but it likely contains the coordinates that define the AOI for the clipping operation.

In summary, this code is defining a dictionary that can be used to specify the parameters for a clipping operation on geospatial data. The parameters include an AOI, which is defined by the `geometry` variable.

In [ ]:
bandmath = {
  "bandmath": {
    "b1": "b1",
    "b2": "b2",
    "b3": "b3",
    "b4": "b4",
    "b5": "(b4 - b3) / (b4 + b3)",
    "pixel_type": "32R",
  }
}

The above Python code is defining a dictionary named `bandmath`. This dictionary is likely used to specify parameters for a band math operation on a multi-band image in a geospatial data processing context.

In the context of geospatial data, "band math" refers to the process of performing mathematical operations on the pixel values of different bands in a multi-band image. Each band in such an image represents a different range of the electromagnetic spectrum and contains a matrix of pixel values.

The `bandmath` dictionary has a single key-value pair. The key is the string "bandmath", and the value is another dictionary.

This inner dictionary has several key-value pairs:

- The keys "b1", "b2", "b3", and "b4" each have a string of the same name as their value. This likely means that these bands should be included in the output as they are, without any mathematical operations performed on them.

- The key "b5" has a string that represents a mathematical operation as its value: "(b4 - b3) / (b4 + b3)". This likely means that a new band "b5" should be created in the output, and its pixel values should be calculated by subtracting the pixel values of band "b3" from those of band "b4", and then dividing the result by the sum of the pixel values of bands "b4" and "b3". This is a common operation in remote sensing used to calculate the Normalized Difference Vegetation Index (NDVI), which is a measure of the amount of live vegetation in an area.

- The key "pixel_type" has the string "32R" as its value. This likely specifies the data type of the pixel values in the output image. "32R" typically represents a 32-bit floating-point data type.

In summary, this code is defining a dictionary that can be used to specify the parameters for a band math operation on a multi-band image. The parameters include the bands to be included in the output, a mathematical operation to be performed to create a new band, and the data type of the pixel values in the output.

In [ ]:
product = [
    {
      "item_ids": [image_ids[0]],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }]

The above Python code is creating a list named `product` that contains a single dictionary. This dictionary is structured to match the expected input for a geospatial data API, likely for ordering a specific type of geospatial data product.

The dictionary within the `product` list contains three keys:

- `item_ids` is a list containing the first element of the `image_ids` list. This is likely a unique identifier for a specific geospatial dataset or image.
- `item_type` is a string that specifies the type of the item. In this case, the item is of type "PSScene", which likely refers to a specific type of geospatial data.
- `product_bundle` is a string that specifies the type of product bundle. In this case, the product bundle is "analytic_udm2", which likely refers to a specific type of data product that includes both analytic data and a Usable Data Mask (UDM2).

In summary, this code is creating a product order for a geospatial data API. The order consists of a single product, which is an analytic data product with a usable data mask for a specific geospatial dataset.

In [ ]:
tool_request = { 
    "name":"San Francisco Clipped Bandmath",
    "products": product,
    "tools": [clip, bandmath],
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

The above Python code is defining a dictionary named `tool_request`. This dictionary is likely used to specify parameters for a request to a geospatial data processing API.

The `tool_request` dictionary contains four key-value pairs:

- `name` is a string that provides a name for the request. In this case, the name is "San Francisco Clipped Bandmath", which suggests that the request involves performing a band math operation on a clipped subset of a geospatial dataset for San Francisco.

- `products` is a list of products to be included in the request. The value of this key is the `product` variable, which is not defined in the selected code but likely contains a list of dictionaries, each representing a specific geospatial data product.

- `tools` is a list of tools to be used in the request. The value of this key is a list containing the `clip` and `bandmath` variables, which are not defined in the selected code but likely contain dictionaries that specify the parameters for a clipping operation and a band math operation, respectively.

- `delivery` is a dictionary that specifies the delivery options for the request. The `single_archive` key is set to `True`, which likely means that all the data products should be delivered in a single archive file. The `archive_type` key is set to "zip", which specifies that the archive file should be in ZIP format.

In summary, this code is defining a dictionary that can be used to specify the parameters for a request to a geospatial data processing API. The parameters include the name of the request, the products to be included, the tools to be used, and the delivery options.

In [ ]:
tool_order_url = place_order(tool_request, auth)

The above Python code is calling the `place_order` function and assigning the result to the variable `tool_order_url`.

The `place_order` function is expected to take two arguments: `tool_request` and `auth`.

- `tool_request` is likely a dictionary that contains the details of the order to be placed. This could include information such as the products to be ordered, the tools to be used for processing, and the delivery options.

- `auth` is likely an object or a tuple that contains the authentication information required to place the order. This could be an API key, a username/password pair, or some other form of credentials.

The `place_order` function is expected to place the order and return the URL for the order. This URL is then stored in the `tool_order_url` variable.

In summary, this line of code is placing an order by calling the `place_order` function with the specified tool request and authentication information, and storing the URL of the placed order.

In [ ]:
#old_url = orders_url+"37580e28-3a1a-4e41-8974-4704faceb529"

### View Results
Now lets review our previous order and download it

In [ ]:
requests.get(order_url, auth=auth).json()['state']

The above Python code is making a GET request to a specified URL and retrieving the 'state' from the JSON response.

Here's a breakdown of what's happening:

- `requests.get(order_url, auth=auth)` is using the `requests` library's `get` method to send a GET request to the URL stored in the `order_url` variable. The `auth` parameter is used for authentication.

- `.json()` is a method that converts the response from the GET request into a Python dictionary. This is useful when the server's response is JSON-formatted, which is common in REST APIs.

- `['state']` is accessing the value associated with the key 'state' in the dictionary. 

In summary, this line of code is sending a GET request to a specified URL with authentication, converting the JSON response into a Python dictionary, and then retrieving the value associated with the 'state' key. This 'state' is likely indicating the status of an order or a process.

In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

The above Python code is making a GET request to a specified URL, converting the response into a Python dictionary, and then extracting a specific piece of data from that dictionary.

Here's a breakdown of what's happening:

- `r = requests.get(order_url, auth=auth)` is using the `requests` library's `get` method to send a GET request to the URL stored in the `order_url` variable. The `auth` parameter is used for authentication. The response from the server is stored in the `r` variable.

- `response = r.json()` is a method that converts the response from the GET request into a Python dictionary. This is useful when the server's response is JSON-formatted, which is common in REST APIs.

- `results = response['_links']['results']` is accessing the value associated with the key 'results' in the dictionary located under the '_links' key in the `response` dictionary. 

In summary, this code is sending a GET request to a specified URL with authentication, converting the JSON response into a Python dictionary, and then retrieving the value associated with the 'results' key under the '_links' key. This 'results' is likely a URL or a list of URLs pointing to the results of a certain operation or request.

In [ ]:
[r['name'] for r in results]

The above Python code is a list comprehension, which is a concise way to create lists in Python. This particular list comprehension is creating a new list that contains the 'name' value from each dictionary in the `results` list.

Here's a breakdown of what's happening:

- `for r in results` is the loop part of the list comprehension. It iterates over each item in the `results` list. Each item is temporarily referred to as `r`.

- `r['name']` is the output expression of the list comprehension. For each iteration of the loop, it accesses the value associated with the key 'name' in the current dictionary (`r`).

So, if `results` is a list of dictionaries, and each dictionary has a key 'name', this list comprehension will create a new list of the 'name' values.

In summary, this line of code is creating a new list that contains the 'name' value from each dictionary in the `results` list.

## Download

### Downloading each asset individually

In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

The above Python code defines a function named `download_results` that downloads files from a list of URLs and saves them to a local directory. The function takes two arguments: `results` and `overwrite`.

Here's a breakdown of what's happening:

- `results_urls = [r['location'] for r in results]` creates a list of URLs from the 'location' key in each dictionary in the `results` list.

- `results_names = [r['name'] for r in results]` creates a list of file names from the 'name' key in each dictionary in the `results` list.

- `print('{} items to download'.format(len(results_urls)))` prints the number of items to be downloaded.

- The `for` loop iterates over pairs of URLs and file names from the `results_urls` and `results_names` lists.

- `path = pathlib.Path(os.path.join('data', name))` creates a `Path` object for the file to be downloaded. The file will be saved in the 'data' directory with its original name.

- The `if` statement checks if the `overwrite` argument is `True` or if the file does not already exist. If either condition is met, the file is downloaded and saved. If not, a message is printed and the file is skipped.

- `r = requests.get(url, allow_redirects=True)` sends a GET request to the URL and allows redirects.

- `path.parent.mkdir(parents=True, exist_ok=True)` creates the parent directory for the file if it does not already exist.

- `open(path, 'wb').write(r.content)` opens the file in write-binary mode and writes the content of the response to it.

In summary, this function downloads files from a list of URLs and saves them to a local directory. If a file already exists, it is either overwritten or skipped based on the `overwrite` argument.

In [ ]:
download_results(results)

The above Python code is calling the `download_results` function with the `results` argument.

The `download_results` function is designed to download files from a list of URLs and save them to a local directory. The `results` argument is expected to be a list of dictionaries, where each dictionary contains at least two keys: 'location' and 'name'. The 'location' key should correspond to the URL of a file to download, and the 'name' key should correspond to the name to use when saving the file locally.

In this case, the `results` list is not defined in the selected code, but it is likely defined elsewhere in the program. The function will iterate over each dictionary in the `results` list, download the file at the URL specified by the 'location' key, and save it locally with the name specified by the 'name' key.

In summary, this line of code is calling the `download_results` function to download a series of files specified by the `results` list and save them locally.

## Subscription API
#### Creating a subscription
[Here](https://developers.planet.com/docs/subscriptions/delivery/) is more info on our various cloud delivery options

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

# set your delivery details
BUCKET_NAME = 'subscriptions_api_demo'
GCS_CREDENTIALS= #64-bit string

The line `GCS_CREDENTIALS=` is incomplete. It's expecting an expression to the right of the `=` sign to assign a value to the `GCS_CREDENTIALS` variable.

In this context, `GCS_CREDENTIALS` is likely meant to hold the credentials for accessing a Google Cloud Storage (GCS) bucket. These credentials are typically provided as a base64-encoded string.

Here's how you might complete the line, assuming you have the base64-encoded credentials stored in a variable called `credentials`:



In [ ]:
GCS_CREDENTIALS = credentials

The line `GCS_CREDENTIALS = credentials` is trying to assign the value of the variable `credentials` to `GCS_CREDENTIALS`. However, if you're seeing an error that "credentials" is not defined, it means that the variable `credentials` has not been assigned a value before this line of code is executed.

In this context, `credentials` is likely meant to hold the credentials for accessing a Google Cloud Storage (GCS) bucket. These credentials are typically provided as a JSON object or a base64-encoded string.

You need to define `credentials` before you can assign its value to `GCS_CREDENTIALS`. 





Replace `"<your-credentials>"` with your actual credentials. Be sure to keep your credentials secure and avoid sharing them publicly.



Remember to replace `credentials` with your actual base64-encoded credentials.



Replace `"<your-credentials>"` with your actual credentials. Be sure to keep your credentials secure and avoid sharing them publicly.

In [ ]:
request = {
    "name": "Tampa_Bay",
    "source": {
        "type": "catalog",
        "parameters": {
            "geometry": {
                "coordinates": [[[-82.775,27.48],[-82.365,27.48],[-82.365,28.07],[-82.775,28.07],[-82.775,27.48]]],
                "type": "Polygon"
            },
            "start_time": "2023-01-01T00:00:00Z",
            "end_time": "2024-03-31T00:00:00Z",
            "rrule": "FREQ=MONTHLY",
            "item_types": ["PSScene"],
            "asset_types": ["ortho_analytic_4b"]
        }
    },
    "delivery": { 
        "type": "google_cloud_storage",
        "parameters": {
            "bucket": BUCKET_NAME,
            "credentials": GCS_CREDENTIALS,
                
        }
    }
}

The above Python code is defining a dictionary named `request`. This dictionary appears to be a configuration for a data request, likely to a REST API. The dictionary has three main keys: `name`, `source`, and `delivery`.

- `name`: This key is assigned the value "Tampa_Bay". This could be the name of the request or the name of the dataset being requested.

- `source`: This key is assigned a dictionary that specifies the details of the data source. The `type` key indicates that the data source is a catalog. The `parameters` key is assigned another dictionary that specifies the parameters of the data request. These parameters include the geographical area of interest (a polygon with specified coordinates), the time range of interest (from January 1, 2023, to March 31, 2024), a rule for repeating the request (monthly), the types of items to be retrieved (PSScene), and the types of assets to be retrieved (ortho_analytic_4b).

- `delivery`: This key is assigned a dictionary that specifies the delivery method for the data. The `type` key indicates that the data should be delivered to Google Cloud Storage. The `parameters` key is assigned another dictionary that specifies the parameters for the delivery. These parameters include the name of the bucket where the data should be stored (`BUCKET_NAME`) and the credentials to be used for accessing the bucket (`GCS_CREDENTIALS`).

In summary, this dictionary is a configuration for a data request. It specifies the name of the request, the details of the data source, and the delivery method for the data.

In [ ]:
def place_subscription(request, auth):
    response = requests.post(subscriptions_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    subscriptions_id = response.json()['id']
    print(subscriptions_id)
    subscription_url = subscriptions_url + '/' + subscriptions_id
    return subscription_url

The function `place_subscription` is trying to use a variable named `subscriptions_url`, but it seems that this variable is not defined in the function or passed as an argument, hence the error.

The `subscriptions_url` is likely the URL of the API endpoint for managing subscriptions. To fix the error, you need to define `subscriptions_url` before it's used or pass it as an argument to the function.

Here's how you can modify the function to take `subscriptions_url` as an argument:





Now, when you call `place_subscription`, you need to provide the `subscriptions_url`:





Replace `"<your-subscriptions-url>"` with your actual subscriptions URL.